In [3]:
import pandas as pd
import numpy as np
import glob
import tqdm
import sqlite3

# from worker import closest_point, match_value, wrapper

# # Read sqlite query results into a pandas DataFrame
# con = sqlite3.connect("data/switrs.sqlite")
# q = """
# SELECT collision_date, longitude, latitude FROM collisions
# WHERE collision_date IS NOT NULL
# AND longitude IS NOT NULL
# AND latitude IS NOT NULL    
# AND collision_date <= '2017-12-31'  -- 2018 is incomplete
# AND collision_date >= '2010-01-01'
# """
# df = pd.read_sql_query(q, con)

# # Round the coordinates, given that NASA only looks at grids of 0.5 x 0.5
# df.latitude = round(df.latitude)
# df.longitude = round(df.longitude)
# df.orig = df.copy()

# # Initially dropped duplicates because that speeds up our investigation
# df.drop_duplicates(inplace=True)
# df.columns = ['collision_date', 'LON', 'LAT']

# Load in the data retrieved from NASA (which are seperate CSV files made up of 4.5 x 4.5 degree Region data)
# See seperate Notebook for loading NASA data
path = 'data/large' # use your path
all_files = glob.glob(path + "/POWER*.csv")

li = []

for filename in all_files:
    df_ap = pd.read_csv(filename, index_col=None, header=10)
    li.append(df_ap)

frame = pd.concat(li, axis=0, ignore_index=True)
frame['DATE'] = frame['YEAR'].map(str) + '-' + pd.to_datetime(frame['DOY'], format='%j').dt.strftime('%m-%d')
frame.drop(['YEAR', 'DOY'], axis=1, inplace=True)
frame.to_csv('data/large/frame.csv')

TypeError: to_sql() missing 1 required positional argument: 'con'

In [526]:
# Create functions for finding the closest point to a point and matching another field to it
from scipy.spatial.distance import cdist
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

# def match_value(df, col1, x, col2, y, col3):
#     """ Match value x from col1 row to value in col2. """
#     return df[(df[col1] == eval(x)) & (df[col3] == y)][col2].values[0]


df1 = frame
# df2 = df.copy()
df2 = pd.read_csv('data/large/intermmediatecsv.csv')
df2['closest'] = [eval(ele) for ele in df2['closest']] 
df1['point'] = [(x, y) for x,y in zip(df1['LAT'], df1['LON'])]
# df2['point'] = [(x, y) for x,y in zip(df2['LAT'], df2['LON'])]

import pandas as pd 
import numpy as np
df2['closest'] = [closest_point(x, list(df1['point'])) for x in tqdm.tqdm(df2['point'])]
# df2['T2M'] = [match_value(df1, 'point', x, 'T2M', y, 'DATE') for x,y in tqdm.tqdm(zip(df2['closest'], df2['collision_date']), total=len(df2['closest']))]
df_1 = df1.set_index(['point', 'DATE'])
df_2 = df2.set_index(['closest', 'collision_date'])
good_keys = df_1.index.intersection(df_2.index)
temp_data = df_1.loc[good_keys].reset_index(level=[0,1]).drop_duplicates()

In [463]:
# Read sqlite query results into a pandas DataFrame
# con = sqlite3.connect("data/switrs.sqlite")
# q = """
# SELECT collision_date, longitude, latitude, weather_1, weather_2 FROM collisions
# WHERE collision_date IS NOT NULL
# AND longitude IS NOT NULL
# AND latitude IS NOT NULL    
# AND collision_date <= '2017-12-31'  -- 2018 is incomplete
# AND collision_date >= '2010-01-01'
# """
# df_origg = pd.read_sql_query(q, con)
df_orig = df_origg.copy()
df_orig.latitude = round(df_orig.latitude*2)/2+0.25
df_orig.longitude = round(df_orig.longitude*2)/2+0.25
df_orig.columns = ['collision_date', 'LON', 'LAT', 'weather_1', 'weather_2']
df_orig['point'] = pd.Series(zip(df_orig['LAT'], df_orig['LON']))

In [506]:
df_orig_1 = df_orig.set_index(['point', 'collision_date'])
df_orig_1.index.rename(['point', 'DATE'], inplace=True)
df_temp = temp_data.set_index(['point', 'DATE'])
good_keys = df_orig_1.index.intersection(df_temp.index)
df_tempp = df_orig_1.loc[good_keys].reset_index(level=[0,1])

In [536]:
temp_data['points'] = list(zip(temp_data['LAT'], temp_data['LON'], temp_data['DATE']))
df_tempp['points'] = list(zip(df_tempp['LAT'], df_tempp['LON'], df_tempp['DATE']))

In [538]:
# Match the original dataset to the already matched data of the unique values, done earlier
def match_value_2(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[(df[col1] == x)][col2].values[0]

df_tempp['T2M'] = [match_value_2(temp_data, 'points', x, 'T2M') for x in tqdm.tqdm(df_tempp['points'])]



  0%|          | 0/247527 [00:00<?, ?it/s]

  0%|          | 8/247527 [00:00<52:51, 78.04it/s]

  0%|          | 19/247527 [00:00<43:00, 95.90it/s]

  0%|          | 31/247527 [00:00<38:52, 106.10it/s]

  0%|          | 43/247527 [00:00<37:04, 111.24it/s]

  0%|          | 56/247527 [00:00<35:36, 115.81it/s]

  0%|          | 68/247527 [00:00<36:16, 113.67it/s]

  0%|          | 80/247527 [00:00<35:58, 114.61it/s]

  0%|          | 92/247527 [00:00<35:38, 115.72it/s]

  0%|          | 105/247527 [00:00<34:24, 119.82it/s]

  0%|          | 118/247527 [00:01<34:38, 119.05it/s]

  0%|          | 130/247527 [00:01<41:16, 99.90it/s] 

  0%|          | 141/247527 [00:01<42:11, 97.72it/s]

  0%|          | 153/247527 [00:01<40:43, 101.22it/s]

  0%|          | 164/247527 [00:01<46:44, 88.20it/s] 

  0%|          | 174/247527 [00:01<47:35, 86.62it/s]

  0%|          | 185/247527 [00:01<45:15, 91.08it/s]

  0%|          | 198/247527 [00:01<40:49, 100.98it/s]

  0%|          | 210/247527 [00:0

  1%|▏         | 3319/247527 [00:29<32:34, 124.93it/s]

  1%|▏         | 3332/247527 [00:29<32:32, 125.09it/s]

  1%|▏         | 3346/247527 [00:29<31:59, 127.23it/s]

  1%|▏         | 3360/247527 [00:29<31:17, 130.03it/s]

  1%|▏         | 3374/247527 [00:29<31:21, 129.76it/s]

  1%|▏         | 3387/247527 [00:29<31:32, 128.97it/s]

  1%|▏         | 3400/247527 [00:29<31:38, 128.58it/s]

  1%|▏         | 3413/247527 [00:29<32:05, 126.77it/s]

  1%|▏         | 3426/247527 [00:30<32:13, 126.27it/s]

  1%|▏         | 3439/247527 [00:30<34:08, 119.16it/s]

  1%|▏         | 3451/247527 [00:30<35:44, 113.83it/s]

  1%|▏         | 3465/247527 [00:30<34:03, 119.44it/s]

  1%|▏         | 3479/247527 [00:30<33:01, 123.17it/s]

  1%|▏         | 3492/247527 [00:30<32:47, 124.06it/s]

  1%|▏         | 3505/247527 [00:30<33:14, 122.33it/s]

  1%|▏         | 3518/247527 [00:30<35:47, 113.64it/s]

  1%|▏         | 3530/247527 [00:30<35:40, 113.98it/s]

  1%|▏         | 3542/247527 [00:31<35:20, 115.0

  3%|▎         | 6863/247527 [00:57<32:44, 122.53it/s]

  3%|▎         | 6876/247527 [00:58<52:36, 76.24it/s] 

  3%|▎         | 6887/247527 [00:58<48:53, 82.04it/s]

  3%|▎         | 6901/247527 [00:58<42:56, 93.39it/s]

  3%|▎         | 6914/247527 [00:58<40:04, 100.07it/s]

  3%|▎         | 6926/247527 [00:58<38:21, 104.55it/s]

  3%|▎         | 6940/247527 [00:58<35:35, 112.64it/s]

  3%|▎         | 6953/247527 [00:58<37:15, 107.62it/s]

  3%|▎         | 6965/247527 [00:58<39:22, 101.82it/s]

  3%|▎         | 6976/247527 [00:59<46:27, 86.30it/s] 

  3%|▎         | 6986/247527 [00:59<53:51, 74.43it/s]

  3%|▎         | 6995/247527 [00:59<56:34, 70.87it/s]

  3%|▎         | 7003/247527 [00:59<1:00:08, 66.66it/s]

  3%|▎         | 7011/247527 [00:59<58:47, 68.18it/s]  

  3%|▎         | 7024/247527 [00:59<48:53, 81.97it/s]

  3%|▎         | 7036/247527 [00:59<44:10, 90.74it/s]

  3%|▎         | 7049/247527 [00:59<40:14, 99.58it/s]

  3%|▎         | 7061/247527 [01:00<39:58, 100.27it/s

  4%|▍         | 10152/247527 [01:28<32:00, 123.59it/s]

  4%|▍         | 10166/247527 [01:29<31:24, 125.97it/s]

  4%|▍         | 10180/247527 [01:29<31:01, 127.51it/s]

  4%|▍         | 10194/247527 [01:29<30:48, 128.42it/s]

  4%|▍         | 10207/247527 [01:29<31:11, 126.79it/s]

  4%|▍         | 10220/247527 [01:29<32:16, 122.54it/s]

  4%|▍         | 10233/247527 [01:29<32:30, 121.67it/s]

  4%|▍         | 10246/247527 [01:29<32:17, 122.48it/s]

  4%|▍         | 10259/247527 [01:29<32:02, 123.39it/s]

  4%|▍         | 10272/247527 [01:29<32:12, 122.80it/s]

  4%|▍         | 10287/247527 [01:29<30:43, 128.68it/s]

  4%|▍         | 10301/247527 [01:30<30:40, 128.89it/s]

  4%|▍         | 10315/247527 [01:30<30:10, 130.99it/s]

  4%|▍         | 10329/247527 [01:30<30:15, 130.64it/s]

  4%|▍         | 10343/247527 [01:30<31:24, 125.86it/s]

  4%|▍         | 10356/247527 [01:30<31:32, 125.35it/s]

  4%|▍         | 10369/247527 [01:30<31:26, 125.68it/s]

  4%|▍         | 10382/247527 [

  5%|▌         | 13583/247527 [01:58<28:51, 135.08it/s]

  5%|▌         | 13598/247527 [01:58<28:23, 137.30it/s]

  5%|▌         | 13613/247527 [01:58<28:03, 138.96it/s]

  6%|▌         | 13628/247527 [01:58<27:54, 139.72it/s]

  6%|▌         | 13643/247527 [01:59<27:49, 140.07it/s]

  6%|▌         | 13658/247527 [01:59<28:11, 138.24it/s]

  6%|▌         | 13672/247527 [01:59<31:38, 123.16it/s]

  6%|▌         | 13685/247527 [01:59<40:07, 97.15it/s] 

  6%|▌         | 13696/247527 [01:59<42:01, 92.74it/s]

  6%|▌         | 13706/247527 [01:59<44:38, 87.31it/s]

  6%|▌         | 13719/247527 [01:59<40:41, 95.77it/s]

  6%|▌         | 13730/247527 [02:00<45:32, 85.56it/s]

  6%|▌         | 13740/247527 [02:00<48:46, 79.87it/s]

  6%|▌         | 13749/247527 [02:00<51:42, 75.36it/s]

  6%|▌         | 13757/247527 [02:00<51:49, 75.19it/s]

  6%|▌         | 13765/247527 [02:00<56:09, 69.38it/s]

  6%|▌         | 13773/247527 [02:00<57:27, 67.80it/s]

  6%|▌         | 13780/247527 [02:00<57:

...

 95%|█████████▌| 236172/247527 [31:19<02:32, 74.46it/s]

 95%|█████████▌| 236180/247527 [31:19<02:46, 67.97it/s]

 95%|█████████▌| 236192/247527 [31:19<02:21, 80.17it/s]

 95%|█████████▌| 236204/247527 [31:19<02:07, 89.09it/s]

 95%|█████████▌| 236214/247527 [31:19<02:08, 88.24it/s]

 95%|█████████▌| 236224/247527 [31:19<02:07, 88.35it/s]

 95%|█████████▌| 236234/247527 [31:19<02:05, 89.75it/s]

 95%|█████████▌| 236245/247527 [31:19<02:01, 92.92it/s]

 95%|█████████▌| 236255/247527 [31:20<01:59, 94.12it/s]

 95%|█████████▌| 236265/247527 [31:20<01:58, 95.08it/s]

 95%|█████████▌| 236275/247527 [31:20<02:11, 85.61it/s]

 95%|█████████▌| 236284/247527 [31:20<02:43, 68.79it/s]

 95%|█████████▌| 236292/247527 [31:20<02:45, 67.88it/s]

 95%|█████████▌| 236300/247527 [31:20<02:43, 68.47it/s]

 95%|█████████▌| 236308/247527 [31:20<02:46, 67.21it/s]

 95%|█████████▌| 236315/247527 [31:21<02:55, 63.90it/s]

 95%|█████████▌| 236322/247527 [31:21<03:00, 61.92it/s]

 95%|█████████▌| 236329/247527 

 96%|█████████▋| 238775/247527 [31:48<01:31, 95.34it/s]

 96%|█████████▋| 238785/247527 [31:48<01:32, 94.16it/s]

 96%|█████████▋| 238796/247527 [31:48<01:30, 96.89it/s]

 96%|█████████▋| 238807/247527 [31:49<01:27, 99.88it/s]

 96%|█████████▋| 238818/247527 [31:49<01:27, 99.49it/s]

 96%|█████████▋| 238828/247527 [31:49<01:31, 95.03it/s]

 96%|█████████▋| 238838/247527 [31:49<01:37, 89.12it/s]

 96%|█████████▋| 238848/247527 [31:49<01:46, 81.63it/s]

 96%|█████████▋| 238857/247527 [31:49<02:04, 69.65it/s]

 97%|█████████▋| 238865/247527 [31:49<02:05, 68.83it/s]

 97%|█████████▋| 238875/247527 [31:50<01:54, 75.44it/s]

 97%|█████████▋| 238885/247527 [31:50<01:46, 81.22it/s]

 97%|█████████▋| 238894/247527 [31:50<01:46, 81.19it/s]

 97%|█████████▋| 238903/247527 [31:50<01:43, 83.26it/s]

 97%|█████████▋| 238913/247527 [31:50<01:39, 86.55it/s]

 97%|█████████▋| 238922/247527 [31:50<01:43, 82.85it/s]

 97%|█████████▋| 238931/247527 [31:50<01:47, 80.32it/s]

 97%|█████████▋| 238940/247527 

 97%|█████████▋| 241153/247527 [32:19<01:20, 79.52it/s]

 97%|█████████▋| 241162/247527 [32:19<01:17, 82.08it/s]

 97%|█████████▋| 241173/247527 [32:19<01:12, 87.95it/s]

 97%|█████████▋| 241182/247527 [32:19<01:14, 85.45it/s]

 97%|█████████▋| 241191/247527 [32:19<01:15, 83.98it/s]

 97%|█████████▋| 241201/247527 [32:19<01:13, 85.94it/s]

 97%|█████████▋| 241212/247527 [32:19<01:08, 92.52it/s]

 97%|█████████▋| 241223/247527 [32:20<01:05, 96.83it/s]

 97%|█████████▋| 241233/247527 [32:20<01:06, 95.33it/s]

 97%|█████████▋| 241243/247527 [32:20<01:12, 86.57it/s]

 97%|█████████▋| 241252/247527 [32:20<01:13, 85.10it/s]

 97%|█████████▋| 241261/247527 [32:20<01:37, 64.32it/s]

 97%|█████████▋| 241269/247527 [32:20<01:41, 61.81it/s]

 97%|█████████▋| 241276/247527 [32:20<01:40, 62.01it/s]

 97%|█████████▋| 241283/247527 [32:20<01:42, 60.98it/s]

 97%|█████████▋| 241290/247527 [32:21<01:53, 54.88it/s]

 97%|█████████▋| 241296/247527 [32:21<02:34, 40.35it/s]

 97%|█████████▋| 241301/247527 

 99%|█████████▊| 243899/247527 [32:49<00:37, 96.67it/s] 

 99%|█████████▊| 243909/247527 [32:49<00:38, 94.45it/s]

 99%|█████████▊| 243920/247527 [32:49<00:36, 98.45it/s]

 99%|█████████▊| 243931/247527 [32:49<00:35, 100.34it/s]

 99%|█████████▊| 243942/247527 [32:50<00:35, 101.81it/s]

 99%|█████████▊| 243954/247527 [32:50<00:34, 104.99it/s]

 99%|█████████▊| 243965/247527 [32:50<00:33, 104.80it/s]

 99%|█████████▊| 243976/247527 [32:50<00:34, 104.21it/s]

 99%|█████████▊| 243987/247527 [32:50<00:33, 105.42it/s]

 99%|█████████▊| 243998/247527 [32:50<00:33, 105.30it/s]

 99%|█████████▊| 244009/247527 [32:50<00:33, 104.76it/s]

 99%|█████████▊| 244021/247527 [32:50<00:32, 107.10it/s]

 99%|█████████▊| 244032/247527 [32:50<00:32, 107.89it/s]

 99%|█████████▊| 244044/247527 [32:50<00:31, 109.15it/s]

 99%|█████████▊| 244055/247527 [32:51<00:31, 108.61it/s]

 99%|█████████▊| 244066/247527 [32:51<00:32, 106.83it/s]

 99%|█████████▊| 244077/247527 [32:51<00:35, 98.57it/s] 

 99%|█████████▊|

100%|█████████▉| 246527/247527 [33:20<00:08, 117.33it/s]

100%|█████████▉| 246539/247527 [33:20<00:08, 117.16it/s]

100%|█████████▉| 246552/247527 [33:20<00:08, 118.92it/s]

100%|█████████▉| 246564/247527 [33:20<00:08, 118.94it/s]

100%|█████████▉| 246577/247527 [33:21<00:07, 120.11it/s]

100%|█████████▉| 246590/247527 [33:21<00:07, 120.18it/s]

100%|█████████▉| 246603/247527 [33:21<00:08, 113.17it/s]

100%|█████████▉| 246615/247527 [33:21<00:08, 112.59it/s]

100%|█████████▉| 246627/247527 [33:21<00:07, 114.04it/s]

100%|█████████▉| 246639/247527 [33:21<00:07, 113.34it/s]

100%|█████████▉| 246652/247527 [33:21<00:07, 115.88it/s]

100%|█████████▉| 246664/247527 [33:21<00:07, 114.40it/s]

100%|█████████▉| 246676/247527 [33:22<00:09, 87.65it/s] 

100%|█████████▉| 246686/247527 [33:22<00:09, 89.22it/s]

100%|█████████▉| 246696/247527 [33:22<00:09, 91.29it/s]

100%|█████████▉| 246706/247527 [33:22<00:08, 92.91it/s]

100%|█████████▉| 246716/247527 [33:22<00:08, 94.63it/s]

100%|█████████▉| 2

In [539]:
# Write the final file to CSV
df_tempp.to_csv('data/large/collisionweather{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [547]:
df_tempp[df_tempp['weather_1'] == 'raining']

point        DATE     LON    LAT weather_1 weather_2  \
10      (33.75, -117.25)  2010-01-13 -117.25  33.75   raining      None   
14      (33.75, -117.25)  2010-01-13 -117.25  33.75   raining      None   
17      (33.75, -117.25)  2010-01-13 -117.25  33.75   raining      None   
20      (38.75, -121.25)  2010-01-01 -121.25  38.75   raining      None   
21      (38.75, -121.25)  2010-01-01 -121.25  38.75   raining      None   
...                  ...         ...     ...    ...       ...       ...   
246915  (40.75, -123.75)  2017-12-02 -123.75  40.75   raining      None   
247018  (39.75, -123.25)  2017-12-19 -123.25  39.75   raining      None   
247019  (39.75, -123.25)  2017-12-19 -123.25  39.75   raining      None   
247170  (40.75, -123.75)  2017-12-20 -123.75  40.75   raining      None   
247526  (41.75, -123.75)  2017-02-16 -123.75  41.75   raining      None   

                              points   T2M  
10      (33.75, -117.25, 2010-01-13)  9.69  
14      (33.75, -117.25, 2010-01-13)  9.69  
17      (33.75, -117.25, 2010-01-13)  9.69  
20      (38.75, -121.25, 2010-01-01)  9.17  
21      (38.75, -121.25, 2010-01-01)  9.17  
...                              ...   ...  
246915  (40.75, -123.75, 2017-12-02)  4.52  
247018  (39.75, -123.25, 2017-12-19)  5.04  
247019  (39.75, -123.25, 2017-12-19)  5.04  
247170  (40.75, -123.75, 2017-12-20)  1.82  
247526  (41.75, -123.75, 2017-02-16)  5.94  

[8827 rows x 8 columns]

In [601]:
# Total Data
collisions_grouped_1 = pd.DataFrame(pd.Series(df_tempp.groupby('DATE').mean()['T2M']))
collisions_grouped_1['count'] = pd.Series(df_tempp['DATE'].value_counts().sort_index())
collisions_grouped_1.columns = ['temp', 'count']
collisions_grouped_1.to_csv('data/large/collisions_grouped_1_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False)

# Conditional on Rain
df_rain = df_tempp[df_tempp['weather_1'] == 'raining']
collisions_grouped_2 = pd.DataFrame(pd.Series(df_rain.groupby('DATE').mean()['T2M']))
collisions_grouped_2['count'] = pd.Series(df_rain['DATE'].value_counts().sort_index())
collisions_grouped_2.columns = ['temp', 'count']
collisions_grouped_2_final = collisions_grouped_2.reindex(pd.DataFrame(pd.Series(df_tempp.groupby('DATE').mean()['T2M'])).index, fill_value=0)
collisions_grouped_2_final.loc[collisions_grouped_2_final['temp'] == 0, 'temp'] = collisions_grouped_1['temp']
collisions_grouped_2_final.to_csv('data/large/collisions_grouped_2_rain_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False)

# Conditional on No Rain
df_no_rain = df_tempp[df_tempp['weather_1'] != 'raining']
collisions_grouped_3 = pd.DataFrame(pd.Series(df_no_rain.groupby('DATE').mean()['T2M']))
collisions_grouped_3['count'] = pd.Series(df_no_rain['DATE'].value_counts().sort_index())
collisions_grouped_3.columns = ['temp', 'count']
collisions_grouped_3_final = collisions_grouped_3.reindex(pd.DataFrame(pd.Series(df_tempp.groupby('DATE').mean()['T2M'])).index, fill_value=0)
collisions_grouped_3_final.loc[collisions_grouped_3_final['temp'] == 0, 'temp'] = collisions_grouped_1['temp']
collisions_grouped_3_final.to_csv('data/large/collisions_grouped_3_no_rain_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), header=False,)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
